In [1]:
%%javascript
require(
        ["notebook/js/outputarea"],
        function (oa) {
            oa.OutputArea.auto_scroll_threshold = -1;
            console.log("Setting auto_scroll_threshold to -1");
        });

<IPython.core.display.Javascript object>

In [2]:
import os, sys, time, resource, re, gc, shutil
from multiprocess import Pool
from functools import partial
from urllib.parse import urlparse, parse_qsl
import matplotlib
matplotlib.use('pgf')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import django
sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()
from django.db.models import Q, F, Sum, Count, FloatField, Case, When, Value, Max
import matplotlib.patches as patches
from scipy.sparse import csr_matrix, find

from scoping.models import *
from tmv_app.models import *

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import normalize
from nltk.stem import SnowballStemmer


import textwrap as tw
from utils.text import *

dtopic_id = 11020

run_id = 665

stat = RunStats.objects.get(pk=run_id)

dt = DynamicTopic.objects.get(pk=dtopic_id)
dtopics = DynamicTopic.objects.filter(run_id=run_id)

In [3]:
tdts = TimeDTopic.objects.filter(dtopic=dt).order_by('period__n')

td_df = pd.DataFrame.from_dict(list(tdts.values('period__n','period__ys','share')))

fig, ax = plt.subplots()

ax.bar(td_df['period__n'],td_df['share'])
vals = ax.get_yticks()
ax.set_yticklabels(['{:,.2%}'.format(x) for x in vals])
ax.set_title(dt.title)


plt.show()

In [38]:
fig, ax = plt.subplots()

def term_plot(dt,ax):
    tts = DynamicTopicTerm.objects.filter(
        topic=dt,
        PY__isnull=False
    )

    cols = 5
    rows = 6
    cells = [[""]*cols for i in range(rows)]
    cell_colours = [[(1,1,1,1)]*cols for i in range(rows)]

    #ax.bar(td_df['period__n'],[1]*cols,color="none")

    #ax.set_xlim(1,6)
    m = tts.aggregate(models.Max('score'))['score__max']
    if m is None:
        m = 0
    min = stat.dyn_win_threshold
    row = []
    
    periods = dt.run_id.periods.all().order_by('n').values_list('n',flat=True)
    
    for col, py in enumerate(periods):
        ytts = tts.filter(
            PY=py,
            score__gt=stat.dyn_win_threshold
        ).order_by('-score')[:cols]
        if ytts.count()==0:
            for row in range(rows-1):
                cell_colours[col][row] = (1,1,1,1)
        for row,t in enumerate(ytts):
            t.scaled_score=np.log(1+t.score)/np.log(1+m)
            ax.text(
                #1/(cols+2)*(col+1),
                col+1,
                1-1/(rows+2)*(row+1),
                t.term.title,
                ha="center",
                va="center",
                bbox={'facecolor':'red', 'alpha':t.scaled_score, 'pad':10}
            )
            cells[col][row] = t.term.title
            cell_colours[col][row] = (1,0,0,t.scaled_score)
            
    topicterms = DynamicTopicTerm.objects.filter(
        topic=dt,
        run_id=run_id,
        score__gt=0.00001,
        PY__isnull=True
    ).order_by('-score')[:12]
    print(topicterms)
    
    tt_cells = [[]]
    tt_cell_colours = [[]]
    for tt in topicterms:
        tt_cells[0].append(tt.term.title)
        tt.scaled_score = np.log(1+tt.score)/np.log(1+m)
        if tt.scaled_score is not None:
            tt_cell_colours[0].append((1,0,0,tt.scaled_score))
        else:
            tt_cell_colours[0].append((1,1,1,1))
        
    return cells, cell_colours, tt_cells, tt_cell_colours

cells, cell_colours, tt_cells, tt_cell_colours = term_plot(dt,ax)
plt.show()
plt.close()
plt.close()
tt_cells

<QuerySet [<DynamicTopicTerm: DynamicTopicTerm object (5006336)>, <DynamicTopicTerm: DynamicTopicTerm object (5005688)>, <DynamicTopicTerm: DynamicTopicTerm object (5005887)>, <DynamicTopicTerm: DynamicTopicTerm object (5015471)>, <DynamicTopicTerm: DynamicTopicTerm object (5010015)>, <DynamicTopicTerm: DynamicTopicTerm object (5007549)>, <DynamicTopicTerm: DynamicTopicTerm object (5015383)>, <DynamicTopicTerm: DynamicTopicTerm object (5012822)>, <DynamicTopicTerm: DynamicTopicTerm object (5003527)>, <DynamicTopicTerm: DynamicTopicTerm object (5017465)>, <DynamicTopicTerm: DynamicTopicTerm object (5004870)>, <DynamicTopicTerm: DynamicTopicTerm object (5009989)>]>


[['biochar',
  'amend',
  'applic',
  'pyrolysi',
  'fertil',
  'compost',
  'properti',
  'manur',
  'increas',
  'straw',
  'treatment',
  'feedstock']]

In [39]:
cell_colours

[[(1, 1, 1, 1), (1, 1, 1, 1), (1, 1, 1, 1), (1, 1, 1, 1), (1, 1, 1, 1)],
 [(1, 1, 1, 1), (1, 1, 1, 1), (1, 1, 1, 1), (1, 1, 1, 1), (1, 1, 1, 1)],
 [(1, 0, 0, 0.06141041455537867),
  (1, 1, 1, 1),
  (1, 1, 1, 1),
  (1, 1, 1, 1),
  (1, 1, 1, 1)],
 [(1, 0, 0, 0.05384628029962752),
  (1, 1, 1, 1),
  (1, 1, 1, 1),
  (1, 1, 1, 1),
  (1, 1, 1, 1)],
 [(1, 0, 0, 0.3738039673836434),
  (1, 0, 0, 0.28395073063314863),
  (1, 0, 0, 0.2694444176864492),
  (1, 0, 0, 0.16867357983457304),
  (1, 0, 0, 0.168224455664034)],
 [(1, 0, 0, 1.0),
  (1, 0, 0, 0.47137723543210464),
  (1, 0, 0, 0.3836715376106785),
  (1, 0, 0, 0.3623794978872554),
  (1, 0, 0, 0.2533274345336113)]]

In [45]:
pgf_with_latex = {
    "text.usetex": True,            # use LaTeX to write all text
    "pgf.rcfonts": False,           # Ignore Matplotlibrc
    "text.latex.unicode": True,
    "pgf.preamble": [
        #r"\usepackage[utf8x]{inputenc}",
        r"\usepackage{xcolor}"
    ],
    "pgf.texsystem" : "xelatex",
    "figure.figsize": [12,7]
}
matplotlib.rcParams.update(pgf_with_latex)

plt.rcParams["figure.figsize"] = (18,12)

gridspec = (7,12)

#plt.close('all')
fig = plt.figure(figsize=(12,7))

max_share = td_df['share'].max()

offset = 0
fs = 9
tpad = 0.02

#Title, words
ax = plt.subplot2grid(gridspec,(0,0),colspan=12,rowspan=1)
ax.set_title(
    dt.title,
    fontdict={
        'fontsize': 16,
        'fontweight': "bold"
    },
    bbox={'facecolor':'white', 'alpha':1, 'pad':6}
)
t = ax.table(
    cellText=[tt_cells[0]],
    cellColours=[tt_cell_colours[0]],  
    bbox=[0.1,0.25,0.8,0.5],
    fontsize=fs,
    cellLoc="center"
)
t.auto_set_font_size(False)
t.set_fontsize(fs)
tcells = [key for key in t._cells]
for cell in tcells:
    t._cells[cell].PAD = tpad
ax.grid(False)
ax.axis('off')


#labels

for name, group in td_df.groupby('period__n'):
    
    print(name)
    
    # share axis
    ax = plt.subplot2grid(gridspec,(name+offset,0),colspan=2,rowspan=1)
    ax.set_xlim(0,max_share*1.1)
    ax.barh(0,group['share'])
    vals = ax.get_xticks()
    ax.set_xticklabels(['{:,.2%}'.format(x) for x in vals])
    plt.yticks([],[])
    #ax.get_yaxis().set_visible(False)
    ax.set_yticks([0])
    ax.set_yticklabels(["AR {}".format(name)])
    
    if name==1:
        ax.set_title("Corpus Share")
    
    if name is not 6:
        ax.get_xaxis().set_visible(False)
        
    
    
    # Terms
    #ax = axs[name-1][1]
    ax = plt.subplot2grid(gridspec,(name+offset,2),colspan=5,rowspan=1)
    t = ax.table(
        cellText=[cells[name-1]],
        cellColours=[cell_colours[name-1]],  
        bbox=[0,0,1,1],
        cellLoc="center",
        fontsize=fs
    )
    t.auto_set_font_size(False)
    t.set_fontsize(fs)
    tcells = [key for key in t._cells]
    for cell in tcells:
        t._cells[cell].PAD = tpad
    ax.grid(False)
    ax.axis('off')
    if name==1:
        ax.set_title("Topic words")
        
    
    
    # Doc
    ax = plt.subplot2grid(gridspec,(name+offset,7),colspan=5,rowspan=1)
    p = TimePeriod.objects.get(timedtopic__dtopic=dt,n=name)
    docs = DocTopic.objects.filter(
        topic__topicdtopic__dynamictopic=dt,
        topic__topicdtopic__score__gt=0.005,
        score__gt=0.005,
        doc__PY__in=p.ys
    )

    docs = docs.annotate(
        topic_combination = F('score') * F('topic__topicdtopic__score')
    ).order_by('-topic_combination')[:2].values(
        'doc__pk','doc__PY','doc__title','topic_combination'
    )
    doc_cells = [[""]*2]
    for i,d in enumerate(docs):
        t = d['doc__title']
        upperchars = [c for c in t if c.isupper()]
        if len(upperchars)/len(t) > 0.5:
            t = "".join([c.lower() if i>0 else c for i,c in enumerate(t)])
        doc_cells[0][i] = tw.fill(tw.dedent(t),width=32)
    
    #doc_cells = [[tw.fill(tw.dedent(d['doc__title']),width=40) for d in docs]]
    t = ax.table(
        cellText=doc_cells,
        bbox=[0,0,1,1],
        fontsize=fs*0.9,
        cellLoc="center"
    )    
    t.auto_set_font_size(False)
    t.set_fontsize(fs)
    tcells = [key for key in t._cells]
    for cell in tcells:
        t._cells[cell].PAD = tpad
    ax.grid(False)
    ax.axis('off')    
    if name==1:
        ax.set_title("Topic documents")
    
    
fig.patch.set_facecolor('#f0f0f0')    
#plt.tight_layout()

plt.savefig(
    "../plots/single_topic_3_{}.pdf".format(dt.id),
    facecolor=fig.get_facecolor(),
    bbox_inches='tight'
)
plt.show()

1
2
3
4
5
6


In [42]:
p = TimePeriod.objects.get(timedtopic__dtopic=dt,n=1)

docs = DocTopic.objects.filter(
    topic__topicdtopic__dynamictopic=dt,
    topic__topicdtopic__score__gt=0.005,
    score__gt=0.005,
    doc__PY__in=p.ys
)

docs = docs.annotate(
    topic_combination = F('score') * F('topic__topicdtopic__score')
).order_by('-topic_combination')[:2].values(
    'doc__pk','doc__PY','doc__title','topic_combination'
)

doc_cells = [[""]*2]
for i,d in enumerate(docs):
    t = d['doc__title']
    upperchars = [c for c in t if c.isupper()]
    #tw.fill(tw.dedent(d['doc__title']),width=40) for d in docs
    if len(upperchars)/len(t) > 0.5:
        t = tw.fill(tw.dedent("".join([c.lower() if i>0 else c for i,c in enumerate(t)])))
        print(t)
    


In [44]:
docs

<QuerySet []>